In [1]:
%load_ext autoreload
%autoreload 2
import notebook_setup
from src.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, REPORTS_DIR, EXTERNAL_DATA_DIR, MODELS_DIR
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"

BREPNET_NPZ_DIR = INTERIM_DATA_DIR / "features" / "brepnet"

2025-09-18 21:31:09.722 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: D:\workspace\projects\freelance\Fusion3DNet


Project root added to path: d:\workspace\projects\freelance\Fusion3DNet


In [2]:
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from typing import List

# Импортируем классы из вашего файла
from src.modeling.vit_brep_ensemble.data_module.dataset import (
    CADItem,
    FusionCADDataset,
    build_brep_standardizer,
    save_stats,
    load_stats,
)

def get_clean_id(filename: str):
    name = Path(filename).stem
    if name.endswith('.prt'):
        name = name[:-4]
    return name

# 1. Укажите пути к вашим директориям с npz файлами
#    (Примерные пути взяты из вашего проекта)
brep_features_dir = Path(INTERIM_DATA_DIR / "features/brepnet")
dino_features_dir = Path(INTERIM_DATA_DIR / "features/dino")
stats_path = Path(INTERIM_DATA_DIR / "features/pooled_brep.json")

# 2. Собираем объекты CADItem, находя общие файлы
brep_files = {p.stem: p for p in brep_features_dir.glob("*.npz")}
dino_files = {p.stem: p for p in dino_features_dir.glob("*.npz")}

# очистим id от лишних суффиксов
brep_files = {get_clean_id(k): v for k, v in brep_files.items()}

common_ids = sorted(brep_files.keys() & dino_files.keys())

print(f"Всего файлов BREP: {len(brep_files)}")
print(f"Всего файлов DINO: {len(dino_files)}")

all_items: List[CADItem] = []
for item_id in common_ids:
    item = CADItem(
        item_id=item_id,
        brep_npz_path=brep_files[item_id],
        dino_path=dino_files[item_id],
    )
    all_items.append(item)

print(f"Найдено {len(all_items)} общих элементов.")

# 3. Разделяем на обучающую и валидационную выборки (например, 80/20)
train_size = int(0.8 * len(all_items))
train_items = all_items[:train_size]
val_items = all_items[train_size:]

print("Вычисление статистики на основе тренировочных данных...")
standardizer = build_brep_standardizer(train_items)
print(f"Сохранение статистики в {stats_path}...")
save_stats(standardizer, stats_path)

# 4. Создаем и сохраняем/загружаем стандартизатор


# Пример получения одного батча
# batch = next(iter(train_loader))
# print("Ключи в батче:", list(batch.keys()))
# print("Размер 'views':", batch["views"].shape)
# print("Размер 'face_matrix':", batch["face_matrix"].shape)
# print("ID элементов:", batch["item_id"])

Всего файлов BREP: 129
Всего файлов DINO: 129
Найдено 128 общих элементов.
Вычисление статистики на основе тренировочных данных...


Вычисление статистики: 100%|██████████| 102/102 [00:00<00:00, 185.11it/s]

Сохранение статистики в D:\workspace\projects\freelance\Fusion3DNet\data\interim\features\pooled_brep.json...


In [3]:
import numpy as np, pathlib as p
f = next(p.Path(INTERIM_DATA_DIR/'features/dino').glob('*.npz'))  # подставьте конкретный файл, на котором падает
with np.load(f) as z:
    print(f, z.files)

D:\workspace\projects\freelance\Fusion3DNet\data\interim\features\dino\42. Ejector-01.npz ['views', 'names']


In [3]:
if stats_path.exists():
    print(f"Загрузка статистики из {stats_path}")
    standardizer = load_stats(stats_path)
else:
    print("Создание и сохранение статистики...")
    standardizer = build_brep_standardizer(train_items)
    save_stats(standardizer, stats_path)

# 5. Создаем экземпляры датасета
train_dataset = FusionCADDataset(
    items=train_items,
    standardizer=standardizer
)
val_dataset = FusionCADDataset(
    items=val_items,
    standardizer=standardizer
)

print(f"Размер обучающего датасета: {len(train_dataset)}")
print(f"Размер валидационного датасета: {len(val_dataset)}")

def custom_collate_fn(batch):
    """
    Собирает батч, оставляя тензоры переменной длины (face_matrix) в виде списка.
    """
    views_list = [item['views'] for item in batch]
    face_matrix_list = [item['face_matrix'] for item in batch]
    item_id_list = [item['item_id'] for item in batch]

    
    views_batch = torch.stack(views_list, dim=0)

    return {
        'views': views_batch,
        'face_matrix': face_matrix_list,
        'item_id': item_id_list
    }

# 6. Используем DataLoader для итерации по данным
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

print(f'Shape dino features: {train_dataset[1]["views"].shape}')

batch = next(iter(train_loader))
print("Ключи в батче:", list(batch.keys()))
print("Размер 'views':", batch["views"].shape)
# 'face_matrix' теперь - это список, выведем размер первого элемента
print("'face_matrix' - это список из", len(batch["face_matrix"]), "тензоров.")
print("Размер первого 'face_matrix' в батче:", batch["face_matrix"][0].shape)
print("ID элементов:", batch["item_id"])

Загрузка статистики из D:\workspace\projects\freelance\Fusion3DNet\data\interim\features\pooled_brep.json
Размер обучающего датасета: 102
Размер валидационного датасета: 26
Shape dino features: torch.Size([8, 384])
Ключи в батче: ['views', 'face_matrix', 'item_id']
Размер 'views': torch.Size([102, 8, 384])
'face_matrix' - это список из 102 тензоров.
Размер первого 'face_matrix' в батче: torch.Size([45, 18])
ID элементов: ['Защелка АК 5', 'Защелка АК 4', 'Защелка АК 6', 'Затвор 9', 'Защелка 5', '43. Extractor-07', 'Защелка 1', 'Защелка 4', '44. Extractor Pin-06', 'Затвор 2', 'Зацеп трубки направляющий 3', 'Защелка АК 3', '44. Extractor Pin-03', 'Кожух 4', 'Защелка АК 8', '42. Ejector-03', 'Защелка 6', 'Камера газовая 10', 'Зацеп трубки направляющий 1', '43. Extractor-03', 'Затвор 4', '44. Extractor Pin-01', 'Камера газовая 3', 'Кожух 3', 'Камера газовая 9', 'Защелка 2', 'Зацеп трубки направляющий 9', 'Затвор 5', '43. Extractor-02', '44. Extractor Pin-10', '42. Silencer Fix-06', '44. Ext

In [7]:
from src.modeling.vit_brep_ensemble import train

train.main()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | dino_encoder | DINOEncoder   | 362 K  | train
1 | brep_encoder | BRepShapeHead | 19.0 K | train
-------------------------------------------------------
381 K     Trainable params
0         Non-trainable params
381 K     Total params
1.525     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Загрузка статистики из D:\workspace\projects\freelance\Fusion3DNet\data\interim\features\pooled_brep.json


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

KeyError: 'views is not a file in the archive'